In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# v3.3 피처삭제 X

## 데이터 불러오기

In [2]:
%pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 2.0 MB/s eta 0:00:00


In [3]:
%pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
import category_encoders as ce
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest, SelectPercentile, SelectFromModel
from sklearn.metrics import f1_score

In [5]:
DATA_PATH = "/content/drive/MyDrive/멋쟁이사자처럼/Final_Project/dataset/"
SEED = 42

In [6]:
train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

In [7]:
train_ft = pd.read_csv(f"{DATA_PATH}train_common_v3.3_피처삭제X_1111.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v3.3_피처삭제X_1111.csv")

train_ft.shape, test_ft.shape, train_target.shape, submit.shape

((14940, 1470), (12225, 1470), (14940, 2), (12225, 2))

## 결측치 처리

In [8]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

## 피처 공학 및 스케일링

In [9]:
# ID 제거
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 1469), (12225, 1469))

In [10]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

,0
주구매지점,4
주구매_대분류,22
주구매_중분류,238


In [11]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft["주구매지점"])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft["주구매지점"])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 1473), (12225, 1473))

In [12]:
enc = ce.count.CountEncoder()
train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

train_ft["주구매_대분류_cnt"] = enc.fit_transform(train_ft[["주구매_대분류"]])
test_ft["주구매_대분류_cnt"] = enc.transform(test_ft[["주구매_대분류"]])

train_ft.shape, test_ft.shape

((14940, 1475), (12225, 1475))

In [13]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 1472), (12225, 1472))

- 가중치 적용

In [14]:
scaler = StandardScaler()

train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,내점일수,구매주기,주말방문비율,평일방문비율,주말방문횟수,평일방문횟수,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,...,공휴일_대분류_영플라자_구매횟수,공휴일_대분류_잡화_구매횟수,공휴일_대분류_케주얼_구두_아동_구매횟수,공휴일_대분류_패션잡화_구매횟수,주구매지점_1,주구매지점_2,주구매지점_3,주구매지점_4,주구매_중분류_cnt,주구매_대분류_cnt
0,-0.369867,0.002987,0.257728,-0.257728,-0.230862,-0.390544,-1.029777,0.001191,0.838272,0.338186,...,-0.162079,-0.286379,-0.209907,-0.213001,1.654066,-0.623175,-0.612523,-0.468181,-0.741478,-1.315250
1,0.144110,-0.356452,-1.008554,1.008554,-0.619841,0.412809,0.323951,-0.390607,0.620171,-0.552996,...,2.389509,-0.286379,-0.209907,4.161199,-0.604571,1.604686,-0.612523,-0.468181,-0.448069,-0.626416
2,1.943028,-0.869935,0.036742,-0.036742,1.616788,1.926821,0.798943,-0.514333,-0.304527,-0.059266,...,-0.162079,-0.286379,-0.209907,-0.213001,-0.604571,-0.623175,1.632592,-0.468181,-0.869072,1.064726
3,3.793345,-1.023980,-0.080558,0.080558,3.172703,4.429574,0.420933,-0.327474,0.008592,-0.135636,...,10.044273,-0.286379,1.301424,-0.213001,1.654066,-0.623175,-0.612523,-0.468181,1.515861,-0.248364
4,0.452496,-0.613193,0.302875,-0.302875,0.838830,0.567300,-0.752532,1.707410,-0.130285,-0.821561,...,-0.162079,4.377687,4.324087,-0.213001,1.654066,-0.623175,-0.612523,-0.468181,-0.847712,1.064726


In [15]:
target = train_target["target"]

## 피처 셀렉션 - SelectFromModel

In [16]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [17]:
xgb = XGBClassifier(random_state=42) # 특성 선택을 하기 위한 모델
model = LogisticRegression(random_state=42) # 학습용 모델

fs_xgb = SelectFromModel(xgb) # 특성 선택에 사용하기 위한 모델 객체를 전달해줘야함.
x = fs_xgb.fit_transform(train_ft, target) # 특성 선택이 완료된 입력 데이터가 ndarray 로 반환
scores = cross_val_score(model, x, target, cv=cv, scoring="f1_macro", n_jobs=-1)
scores.mean()

0.7149830146365065

In [18]:
rfc = RandomForestClassifier(random_state=42) # 특성 선택을 하기 위한 모델
model = LogisticRegression(random_state=42) # 학습용 모델

fs_rf = SelectFromModel(rfc) # 특성 선택에 사용하기 위한 모델 객체를 전달해줘야함.
x = fs_rf.fit_transform(train_ft, target) # 특성 선택이 완료된 입력 데이터가 ndarray 로 반환
scores = cross_val_score(model, x, target, cv=cv, scoring="f1_macro", n_jobs=-1)
scores.mean()

0.7144155946325675

In [19]:
xgb_best_cols = fs_xgb.get_feature_names_out()
len(xgb_best_cols)

574

In [20]:
rf_best_cols = fs_rf.get_feature_names_out()
len(rf_best_cols)

365

## Logistic Regression

- XGBoost로 피처 셀렉션

In [21]:
def objective(trial):
    hp = {
        "C": trial.suggest_float("C", 1e-4, 10.0, log=True),  # 규제 강도
        "penalty": "l2",  # L2 규제로 고정
        "solver": trial.suggest_categorical("solver", ["lbfgs", "liblinear", "sag", "saga"]),
        "max_iter": trial.suggest_int("max_iter", 50, 500, step=50)  # 반복 횟수
    }
    cv = KFold(5, shuffle=True, random_state=SEED)
    model = LogisticRegression(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft[xgb_best_cols], target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

sampler = optuna.samplers.TPESampler(seed=SEED) # 대체 모델 역할은 sampler 객체

# study 객체
study = optuna.create_study(
    direction="maximize",
    sampler = sampler
)

study.optimize(objective, n_trials=100)   # optimze 메서드에 콜백 함수(objective) 전달

[I 2024-11-11 05:34:38,339] A new study created in memory with name: no-name-67019a69-b225-497d-90a5-605b3ad4ff61
[I 2024-11-11 05:34:44,333] Trial 0 finished with value: 0.7161863200967151 and parameters: {'C': 0.0074593432857265485, 'solver': 'lbfgs', 'max_iter': 100}. Best is trial 0 with value: 0.7161863200967151.
[I 2024-11-11 05:34:46,978] Trial 1 finished with value: 0.7031411912246315 and parameters: {'C': 0.00019517224641449495, 'solver': 'lbfgs', 'max_iter': 500}. Best is trial 0 with value: 0.7161863200967151.
[I 2024-11-11 05:38:01,410] Trial 2 finished with value: 0.713716316049088 and parameters: {'C': 1.452824663751602, 'solver': 'saga', 'max_iter': 300}. Best is trial 0 with value: 0.7161863200967151.
[I 2024-11-11 05:38:15,559] Trial 3 finished with value: 0.7157898478948141 and parameters: {'C': 0.01444525102276306, 'solver': 'liblinear', 'max_iter': 200}. Best is trial 0 with value: 0.7161863200967151.
[I 2024-11-11 05:38:19,707] Trial 4 finished with value: 0.715224

In [22]:
# 최적의 하이퍼파라미터 출력
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

Best hyperparameters: {'C': 0.00048637842387982456, 'solver': 'liblinear', 'max_iter': 50}
Best F1 Score: 0.7253699717954318


In [26]:
log_xgb_params = {'C': 0.00048637842387982456,
                  'solver': 'liblinear',
                  'max_iter': 50,
                  'penalty':'l2'}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리 (선택된 피처만 사용)
    x_train = train_ft.iloc[tri][xgb_best_cols]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai][xgb_best_cols]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    logistic_model_xgb = LogisticRegression(**log_xgb_params)
    logistic_model_xgb.fit(x_train, y_train)

    # 모델 저장
    models.append(logistic_model_xgb)

    # 예측 및 F1 매크로 스코어 계산
    pred = logistic_model_xgb.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

Mean F1 Macro Score: 0.7262212451799022


In [ ]:
logistic_model_xgb = LogisticRegression(**log_xgb_params)
logistic_model_xgb.fit(train_ft[xgb_best_cols],target)

pred = logistic_model_xgb.predict_proba(test_ft[xgb_best_cols])[:, 1]

submit["target"] = pred
submit.to_csv("logistic_model_xgb.csv",index=False)

In [ ]:
# # 모델 학습
# model = LogisticRegression(random_state=SEED, **study.best_params)
# model.fit(train_ft[best_cols],target)

# pred = model.predict(test_ft[best_cols])

# submit["target"] = pred
# submit.to_csv(f"{DATA_PATH}6_v3.2_Logistic.csv",index=False)

- RandomForestClassifier로 피처 셀렉션

In [24]:
def objective(trial):
    hp = {
        "C": trial.suggest_float("C", 1e-4, 10.0, log=True),  # 규제 강도
        "penalty": "l2",  # L2 규제로 고정
        "solver": trial.suggest_categorical("solver", ["lbfgs", "liblinear", "sag", "saga"]),
        "max_iter": trial.suggest_int("max_iter", 50, 500, step=50)  # 반복 횟수
    }
    cv = KFold(5, shuffle=True, random_state=SEED)
    model = LogisticRegression(**hp, random_state=SEED)
    score = cross_val_score(model, train_ft[rf_best_cols], target, cv=cv, scoring="f1_macro", n_jobs=-1).mean()
    return score

sampler = optuna.samplers.TPESampler(seed=SEED) # 대체 모델 역할은 sampler 객체

# study 객체
study = optuna.create_study(
    direction="maximize",
    sampler = sampler
)

study.optimize(objective, n_trials=100)   # optimze 메서드에 콜백 함수(objective) 전달

[I 2024-11-11 06:15:21,198] A new study created in memory with name: no-name-3180760d-a20d-4efd-8100-942183800520
[I 2024-11-11 06:15:24,427] Trial 0 finished with value: 0.7150373897749273 and parameters: {'C': 0.0074593432857265485, 'solver': 'lbfgs', 'max_iter': 100}. Best is trial 0 with value: 0.7150373897749273.
[I 2024-11-11 06:15:27,774] Trial 1 finished with value: 0.7009958034470959 and parameters: {'C': 0.00019517224641449495, 'solver': 'lbfgs', 'max_iter': 500}. Best is trial 0 with value: 0.7150373897749273.
[I 2024-11-11 06:17:34,561] Trial 2 finished with value: 0.7122994962381288 and parameters: {'C': 1.452824663751602, 'solver': 'saga', 'max_iter': 300}. Best is trial 0 with value: 0.7150373897749273.
[I 2024-11-11 06:17:46,779] Trial 3 finished with value: 0.7139803700053353 and parameters: {'C': 0.01444525102276306, 'solver': 'liblinear', 'max_iter': 200}. Best is trial 0 with value: 0.7150373897749273.
[I 2024-11-11 06:17:49,724] Trial 4 finished with value: 0.71392

In [25]:
# 최적의 하이퍼파라미터 출력
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

Best hyperparameters: {'C': 0.0006140236436258448, 'solver': 'liblinear', 'max_iter': 150}
Best F1 Score: 0.7219254243015749


In [27]:
log_rf_params = {'C': 0.0006140236436258448,
                 'solver': 'liblinear',
                 'max_iter': 150,
                 'penalty':'l2'}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리 (선택된 피처만 사용)
    x_train = train_ft.iloc[tri][rf_best_cols]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai][rf_best_cols]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    logistic_model_rf = LogisticRegression(**log_rf_params)
    logistic_model_rf.fit(x_train, y_train)

    # 모델 저장
    models.append(logistic_model_rf)

    # 예측 및 F1 매크로 스코어 계산
    pred = logistic_model_rf.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

Mean F1 Macro Score: 0.7244022447608789


In [ ]:
logistic_model_rf = LogisticRegression(**log_rf_params)
logistic_model_rf.fit(train_ft[rf_best_cols],target)

pred = logistic_model_rf.predict_proba(test_ft[rf_best_cols])[:, 1]

submit["target"] = pred
submit.to_csv("logistic_model_rf.csv",index=False)

In [ ]:
# # 모델 학습
# model = LogisticRegression(random_state=SEED, **study.best_params)
# model.fit(train_ft[rf_best_cols],target)

# pred = model.predict(test_ft[rf_best_cols])

# submit["target"] = pred
# submit.to_csv(f"{DATA_PATH}6_v3.2.02_Logistic.csv",index=False)

## LDA

- XGBoost로 피처 셀렉션

In [28]:
def objective(trial):
    # Optuna가 선택할 수 있는 하이퍼파라미터 정의
    solver = trial.suggest_categorical('solver', ['svd', 'lsqr', 'eigen'])

    # 'lsqr'와 'eigen'에서만 shrinkage 하이퍼파라미터가 사용되므로 조건 추가
    if solver in ['lsqr', 'eigen']:
        shrinkage = trial.suggest_float('shrinkage', 0.0, 1.0)
    else:
        shrinkage = None

    # n_components 하이퍼파라미터: 차원 수를 1부터 X의 특성 개수까지
    n_components = trial.suggest_int('n_components', 1, min(train_ft[xgb_best_cols].shape[1], len(np.unique(target)) - 1))

    # LDA 모델 정의
    lda = LinearDiscriminantAnalysis(solver=solver, shrinkage=shrinkage, n_components=n_components)

    score = cross_val_score(lda, train_ft[xgb_best_cols], target, cv=5, scoring="f1_macro", n_jobs=-1).mean()
    return score

# Optuna study 설정 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

[I 2024-11-11 06:42:16,063] A new study created in memory with name: no-name-67f0d6b3-d26c-4aa0-8bd8-37b42bea2e6d
[I 2024-11-11 06:42:20,158] Trial 0 finished with value: 0.7240218373496148 and parameters: {'solver': 'lsqr', 'shrinkage': 0.5513421037476925, 'n_components': 1}. Best is trial 0 with value: 0.7240218373496148.
[I 2024-11-11 06:42:33,325] Trial 1 finished with value: 0.7142166313274505 and parameters: {'solver': 'svd', 'n_components': 1}. Best is trial 0 with value: 0.7240218373496148.
[I 2024-11-11 06:42:37,186] Trial 2 finished with value: 0.72402347799233 and parameters: {'solver': 'lsqr', 'shrinkage': 0.47181904966305943, 'n_components': 1}. Best is trial 2 with value: 0.72402347799233.
[I 2024-11-11 06:42:50,103] Trial 3 finished with value: 0.7142166313274505 and parameters: {'solver': 'svd', 'n_components': 1}. Best is trial 2 with value: 0.72402347799233.
[I 2024-11-11 06:42:53,966] Trial 4 finished with value: 0.7228257840190448 and parameters: {'solver': 'lsqr', 

In [29]:
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

Best hyperparameters: {'solver': 'lsqr', 'shrinkage': 0.5088845338774974, 'n_components': 1}
Best F1 Score: 0.7246109969277355


In [30]:
lda_xgb_params = {'solver': 'lsqr', 'shrinkage': 0.5088845338774974, 'n_components': 1}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리 (선택된 피처만 사용)
    x_train = train_ft.iloc[tri][xgb_best_cols]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai][xgb_best_cols]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    lda_model_xgb = LinearDiscriminantAnalysis(**lda_xgb_params)
    lda_model_xgb.fit(x_train, y_train)

    # 모델 저장
    models.append(lda_model_xgb)

    # 예측 및 F1 매크로 스코어 계산
    pred = lda_model_xgb.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

Mean F1 Macro Score: 0.7253460775540519


In [ ]:
lda_model_xgb = LinearDiscriminantAnalysis(**lda_xgb_params)
lda_model_xgb.fit(train_ft[xgb_best_cols],target)

pred = lda_model_xgb.predict_proba(test_ft[xgb_best_cols])[:, 1]

submit["target"] = pred
submit.to_csv("lda_model_xgb.csv",index=False)

In [ ]:
# model = LinearDiscriminantAnalysis(**study.best_params)
# model.fit(train_ft[xgb_best_cols],target)

# pred = model.predict(test_ft[xgb_best_cols])

# submit["target"] = pred
# submit.to_csv(f"{DATA_PATH}6_v3.2_LDA.csv",index=False)

- RandomForestClassifier로 피처 셀렉션

In [32]:
def objective(trial):
    # Optuna가 선택할 수 있는 하이퍼파라미터 정의
    solver = trial.suggest_categorical('solver', ['svd', 'lsqr', 'eigen'])

    # 'lsqr'와 'eigen'에서만 shrinkage 하이퍼파라미터가 사용되므로 조건 추가
    if solver in ['lsqr', 'eigen']:
        shrinkage = trial.suggest_float('shrinkage', 0.0, 1.0)
    else:
        shrinkage = None

    # n_components 하이퍼파라미터: 차원 수를 1부터 X의 특성 개수까지
    n_components = trial.suggest_int('n_components', 1, min(train_ft[rf_best_cols].shape[1], len(np.unique(target)) - 1))

    # LDA 모델 정의
    lda = LinearDiscriminantAnalysis(solver=solver, shrinkage=shrinkage, n_components=n_components)

    score = cross_val_score(lda, train_ft[rf_best_cols], target, cv=5, scoring="f1_macro", n_jobs=-1).mean()
    return score

# Optuna study 설정 및 최적화 실행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=200)

[I 2024-11-11 07:06:58,995] A new study created in memory with name: no-name-cf909c89-c7e7-4510-b1a0-81d326a94806
[I 2024-11-11 07:07:04,203] Trial 0 finished with value: 0.7207085607458504 and parameters: {'solver': 'lsqr', 'shrinkage': 0.4034606222878657, 'n_components': 1}. Best is trial 0 with value: 0.7207085607458504.
[I 2024-11-11 07:07:07,409] Trial 1 finished with value: 0.7167302398620929 and parameters: {'solver': 'eigen', 'shrinkage': 0.03004091323140856, 'n_components': 1}. Best is trial 0 with value: 0.7207085607458504.
[I 2024-11-11 07:07:09,484] Trial 2 finished with value: 0.6355754787913744 and parameters: {'solver': 'lsqr', 'shrinkage': 0.9879216935494545, 'n_components': 1}. Best is trial 0 with value: 0.7207085607458504.
[I 2024-11-11 07:07:11,988] Trial 3 finished with value: 0.7203408208114428 and parameters: {'solver': 'eigen', 'shrinkage': 0.49169981932033313, 'n_components': 1}. Best is trial 0 with value: 0.7207085607458504.
[I 2024-11-11 07:07:17,578] Trial 

In [33]:
print("Best hyperparameters:", study.best_params)
print("Best F1 Score:", study.best_value)

Best hyperparameters: {'solver': 'lsqr', 'shrinkage': 0.32247003082505904, 'n_components': 1}
Best F1 Score: 0.7213737065832213


In [34]:
lda_rf_params = {'solver': 'lsqr', 'shrinkage': 0.32247003082505904, 'n_components': 1}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리 (선택된 피처만 사용)
    x_train = train_ft.iloc[tri][rf_best_cols]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai][rf_best_cols]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    lda_model_rf = LinearDiscriminantAnalysis(**lda_rf_params)
    lda_model_rf.fit(x_train, y_train)

    # 모델 저장
    models.append(lda_model_rf)

    # 예측 및 F1 매크로 스코어 계산
    pred = lda_model_rf.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))

Mean F1 Macro Score: 0.7225667672772422


In [ ]:
lda_model_rf = LinearDiscriminantAnalysis(**lda_rf_params)
lda_model_rf.fit(train_ft[rf_best_cols],target)

pred = lda_model_rf.predict_proba(test_ft[rf_best_cols])[:, 1]

submit["target"] = pred
submit.to_csv("lda_model_rf.csv",index=False)

In [ ]:
# model = LinearDiscriminantAnalysis(**study.best_params)
# model.fit(train_ft[rf_best_cols],target)

# pred = model.predict(test_ft[rf_best_cols])

# submit["target"] = pred
# submit.to_csv(f"{DATA_PATH}6_v3.2.02_LDA.csv",index=False)

## 스태킹

In [37]:
from sklearn.ensemble import StackingClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer

xgb_features = train_ft[xgb_best_cols]
rf_features = train_ft[rf_best_cols]

# 각각 모델에 대해 피처 셀렉션을 반영한 파이프라인 구성
estimators = [
    ("log_xgb", Pipeline([
        ("select_xgb_features", FunctionTransformer(lambda x: x[xgb_best_cols], validate=False)),
        ("log_xgb_model", LogisticRegression(**log_xgb_params, random_state=SEED))
    ])),
    ("log_rf", Pipeline([
        ("select_rf_features", FunctionTransformer(lambda x: x[rf_best_cols], validate=False)),
        ("log_rf_model", LogisticRegression(**log_xgb_params, random_state=SEED))
    ])),
    ("lda_xgb", Pipeline([
        ("select_xgb_features", FunctionTransformer(lambda x: x[xgb_best_cols], validate=False)),
        ("lda_xgb_model", LinearDiscriminantAnalysis(**lda_xgb_params))
    ])),
    ("lda_rf", Pipeline([
        ("select_rf_features", FunctionTransformer(lambda x: x[rf_best_cols], validate=False)),
        ("lda_rf_model", LinearDiscriminantAnalysis(**lda_rf_params))
    ]))
]

params = {
    "estimators": estimators,
    "final_estimator": LogisticRegression(random_state=SEED),
    "cv": cv,
    "n_jobs": -1
}

model = StackingClassifier(**params)

# cross-validation 평가
scores = cross_val_score(model, train_ft, target, cv=cv, scoring="f1_macro", n_jobs=-1)
print(scores.mean())

0.7199200922054594
